In [1]:
import torch

from transformer_lens import HookedTransformer
from transformer_lens.utils import get_act_name

torch.set_grad_enabled(False) 

In [2]:
model = HookedTransformer.from_pretrained("gpt2")

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer


In [6]:
prompt = "I want to kill you because you're such a"

In [8]:
steer_positive_prompt = " Bread"
steer_negative_prompt = " "

In [9]:
tokens = model.to_tokens(prompt)
steer_positive_tokens = model.to_tokens(steer_positive_prompt)
steer_negative_tokens = model.to_tokens(steer_negative_prompt)

In [10]:
steer_positive_tokens

tensor([[50256, 28731]])

In [11]:
steer_negative_tokens

tensor([[50256,   220]])

In [29]:
# hook_name = get_act_name("attn", 6)
hook_name = "blocks.6.hook_resid_pre"

In [30]:
_, steer_positive_activations = model.run_with_cache(steer_positive_tokens)

In [31]:
target_positive_activations = steer_positive_activations[hook_name]

In [32]:
target_positive_activations.shape

torch.Size([1, 2, 768])

In [34]:
def add_resid_pre_activations(activations, hook):
    # n_tokens = target_positive_activations.shape[1]
    # set_trace()
    activations[:, :2, :] += target_positive_activations * 15
    return activations

In [35]:
# def add_activations(activations, hook):
#     n_tokens = target_positive_activations[1]
#     activations[:, :n_tokens, :] = target_positive_activations
#     return activations
#     # for attn
#     # n_tokens = target_activations.shape[-1]
#     # activations[:, :, :n_tokens, :n_tokens] += target_activations * 10
    
# #     n_tokens = target_activations.shape[1]
# #     activations[:, :n_tokens, :] += target_activations
    
#     # set_trace()
#     # return activations

In [36]:
import torch
import torch.nn.functional as F
from einops import rearrange

from transformer_lens import utils

from IPython.core.debugger import set_trace

In [37]:
def logit_to_pred_token(logits):
    # set_trace()
    pred_logits = logits[0, -1, :]
    probs = F.softmax(pred_logits, dim=-1)
    pred_token = torch.argmax(pred_logits, dim=-1)
    return pred_token

In [38]:
n_new_tokens = 20
generated_tokens = tokens

In [39]:
model.reset_hooks()

In [40]:
# model.add_hook(hook_name, add_resid_pre_activations)

In [41]:
# generated_tokens = model.generate(tokens, max_new_tokens=20)

In [42]:
model.to_string(generated_tokens)

["<|endoftext|>I want to kill you because you're such a"]

In [43]:
# for _ in range(n_new_tokens):
#     modified_logits = model.run_with_hooks(
#         generated_tokens,
#         fwd_hooks=[(hook_name, add_resid_pre_activations)]
#     )
#     pred_token = logit_to_pred_token(modified_logits)
#     generated_tokens = torch.cat([
#         generated_tokens,
#         pred_token.unsqueeze(0).unsqueeze(0)
#     ], dim=-1)
    
#     print(model.to_string(generated_tokens))

In [44]:
for _ in range(n_new_tokens):
    modified_logits = model.run_with_hooks(
        generated_tokens,
        fwd_hooks=[(hook_name, add_resid_pre_activations)]
    )
    pred_token = logit_to_pred_token(modified_logits)
    # set_trace()
    generated_tokens = torch.cat([
        generated_tokens,
        # rearrange(pred_token, "n -> 1 1 n")
        pred_token.unsqueeze(0).unsqueeze(0)
    ], dim=-1)
    
    print(model.to_string(generated_tokens))

["<|endoftext|>I want to kill you because you're such a fucking"]
["<|endoftext|>I want to kill you because you're such a fucking idiot"]
["<|endoftext|>I want to kill you because you're such a fucking idiot."]
["<|endoftext|>I want to kill you because you're such a fucking idiot.\n"]
["<|endoftext|>I want to kill you because you're such a fucking idiot.\n\n"]
["<|endoftext|>I want to kill you because you're such a fucking idiot.\n\nI"]
["<|endoftext|>I want to kill you because you're such a fucking idiot.\n\nI want"]
["<|endoftext|>I want to kill you because you're such a fucking idiot.\n\nI want to"]
["<|endoftext|>I want to kill you because you're such a fucking idiot.\n\nI want to kill"]
["<|endoftext|>I want to kill you because you're such a fucking idiot.\n\nI want to kill you"]
["<|endoftext|>I want to kill you because you're such a fucking idiot.\n\nI want to kill you because"]
["<|endoftext|>I want to kill you because you're such a fucking idiot.\n\nI want to kill you because 

In [ ]:
generated_tokens

In [87]:
default_kwargs = {
    "temperature": 1.0,
    "top_p": 0.3,
    "freq_penalty": 1.0,
    # "num_comparisons": 3,
    # "tokens_to_generate": 50,
    # "seed": 0, # For reproducibility
}

In [88]:
model.generate(
    input="I hate you because",
    max_new_tokens=40,
    verbose=False,
    **default_kwargs)

"I hate you because I'm a woman. I hate you because I'm a woman.\n\nI don't know what to say about this, but the reason that we're talking about women is that we're not"